#### Student Name: Dwight Devens
#### Student ID: A15711217

# CSE 190 Final Project

### Suggested Project 1

### Composing Melody Using RNN with Attention



In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import wavfile
from numpy.linalg import svd
from scipy.stats.mstats import gmean
from matplotlib import rcParams
import scipy
import os
import sys
import glob
import pickle
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.layers import Lambda

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


filePathToWeights = 'weights.hdf5'
lengthOfSequenceProduced = 64

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


# Get the data ready

In [2]:
def constructMelodies(notes, pitches, n_vocab):
    # Map notes to integers
    noteToIntegerDict = dict((note, number) for number, note in enumerate(pitches))

    numberOfNotesForPrediction = 8
    networkInput = []
    networkOutput = []
    for i in range(0, len(notes) - numberOfNotesForPrediction, 1):
        # Input
        inputSequence = notes[i:i + numberOfNotesForPrediction]
        networkInput.append([noteToIntegerDict[char] for char in inputSequence])
        # Output
        outputSequence = notes[i + numberOfNotesForPrediction]
        networkOutput.append(noteToIntegerDict[outputSequence])
    # Store the length
    theNetworkPatttern = len(networkInput)

    # Had to change shape for LSTM layers or it wasn't working
    inputAfterNormalization = np.reshape(networkInput, (theNetworkPatttern, numberOfNotesForPrediction, 1))
    inputAfterNormalization = inputAfterNormalization / float(n_vocab)

    return (networkInput, inputAfterNormalization)

# Get the next note

In [3]:
def generate_next_note(model, networkInput, pitches, n_vocab):
    # Changed to a random start point to try to get better output
    randomStartingPoint = np.random.randint(0, len(networkInput)-1)
    # Map the integers back to the notes
    integerToNoteDict = dict((number, note) for number, note in enumerate(pitches))

    thePattern = networkInput[randomStartingPoint]
    # Prediction to return
    actualPrediction = []

    # Note generation
    for note_index in range(lengthOfSequenceProduced):
        # Reshape and normalize
        predictionInput = np.reshape(thePattern, (1, len(thePattern), 1))
        predictionInput = predictionInput / float(n_vocab)

        prediction = model.predict(predictionInput, verbose=0)
        #print("prediction = ", prediction)

        index = np.argmax(prediction)
        
        # Wrote to catch error, so far error is gone, delete later if needed
        if index in integerToNoteDict:
            result = integerToNoteDict[index]
        else:
            result = 'error'
        # Add the prediction
        actualPrediction.append(result)
        # Then add to the pattern
        thePattern.append(index)
        thePattern = thePattern[1:len(thePattern)]

    return actualPrediction

# Network construction

In [4]:
def constructNetwork(networkInput, n_vocab):
    
    model = Sequential()
    model.add(LSTM(512, input_shape=(networkInput.shape[1], networkInput.shape[2]),
                   recurrent_dropout=0.3, return_sequences=True))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    
    # 2. A batch normalization layer. 
    # Keras documentation - "Layer that normalizes its inputs. Batch normalization 
    # applies a transformation that maintains the mean output close to 0 and the output 
    # standard deviation close to 1.""
    # Example: Args look optional, just instantiate for now
    model.add(BatchNorm())
    
    # 3. A layer which drops 3/10 of the units. 
    # Keras documentation - "The Dropout layer randomly sets input units to 0 with a 
    # frequency of rate at each step during training time, 
    # which helps prevent overfitting. 
    # Inputs not set to 0 are scaled up by 1/(1 - rate) such that the sum over all 
    # inputs is unchanged.""
    # Example: tf.keras.layers.Dropout(rate, noise_shape=None, seed=None, **kwargs)
    model.add(Dropout(0.3))
    
    # 4. A fully connected layer with 256 units of output. 
    # Keras documentation - "Just your regular densely-connected NN layer."
    # Example: model.add(tf.keras.layers.Dense(32))
    model.add(Dense(256))
    
    # 5. A ReLU activation layer. 
    # Keras documentatino - "Applies an activation function to an output.
    # Arguments - activation: Activation function, such as tf.nn.relu, 
    # or string name of built-in activation function, such as "relu"."
    # Example: layer = tf.keras.layers.Activation('relu')
    model.add(Activation('relu'))
    
    # 6. A batch normalization layer. 
    # Keras documentation - "Layer that normalizes its inputs. Batch normalization 
    # applies a transformation that maintains the mean output close to 0 and the output 
    # standard deviation close to 1.""
    # Example: Args look optional, just instantiate for now
    model.add(BatchNorm())
    
    # 7. A layer which drops 3/10 of the units. 
    # Keras documentation - "The Dropout layer randomly sets input units to 0 with a 
    # frequency of rate at each step during training time, 
    # which helps prevent overfitting. 
    # Inputs not set to 0 are scaled up by 1/(1 - rate) such that the sum over all 
    # inputs is unchanged.""
    # Example: tf.keras.layers.Dropout(rate, noise_shape=None, seed=None, **kwargs)
    model.add(Dropout(0.3))
    
    # 8. A fully connected layer with number of units of output equal to the vocabulary 
    # space of the input. 
    # Keras documentation - "Just your regular densely-connected NN layer."
    # Example: model.add(tf.keras.layers.Dense(32))
    # but this time need vocab space of input, which = n_vocab  
    model.add(Dense(n_vocab))
    
    # 9. A softmax activation layer which uses a temperature of .6 
    # (Note, you may need to define this as two separate layers in Keras, 
    # using the definition of temperature for softmax)
    
    # Keras documentation - "Softmax converts a vector of values to a probability 
    # distribution. The elements of the output vector are in range (0, 1) and sum to 1.
    # Each vector is handled independently. The axis argument sets which axis of the 
    # input the function is applied along. Softmax is often used as the activation 
    # for the last layer of a classification network because the result could be 
    # interpreted as a probability distribution. The softmax of each vector x is 
    # computed as exp(x) / tf.reduce_sum(exp(x)). The input values in are the log-odds 
    # of the resulting probability.
    # Arguments - x : Input tensor. axis: Integer, axis along which the softmax 
    # normalization is applied."
    model.add(Activation('softmax'))
    
    # After creating your network, compile the model with categorical cross entropy loss 
    # and an optimizer of your choice. 
    # Example: model.compile(loss='categorical_crossentropy', optimizer=opt)
    
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights(filePathToWeights)

    return model

# Create the output

In [5]:
def buildMidiOutputFile(actualPrediction):
    bufferBetween = 0
    # Leave this at 0.5 for now
    bufferAmount = 0.5
    notesForTheOutput = []
    
    # For each pattern
    for thePattern in actualPrediction:
        # If the pattern is a note
        if ('.' in thePattern) or thePattern.isdigit():    
            # Create new note and append to output
            newNote = note.Note(thePattern)
            newNote.storedInstrument = instrument.Piano()
            newNote.bufferBetween = bufferBetween
            notesForTheOutput.append(newNote)  
        # Else it is a chord
        else:
            chordNotes = thePattern.split('.')
            notes = []
            # Then for each note in the chord
            for thisNote in chordNotes:
                # Add the notes together
                newNote = note.Note(int(thisNote))
                newNote.storedInstrument = instrument.Piano()
                notes.append(newNote)
            # And create a new chord and append the output
            newChord = chord.Chord(notes)
            newChord.bufferBetween = bufferBetween
            notesForTheOutput.append(newChord)

        # Add a buffer between the notes each time
        bufferBetween += bufferAmount
        
    # Construct the midi output file
    midiOutput = stream.Stream(notesForTheOutput)
    midiOutput.write('midi', fp='outputFile.mid')

# Run the whole project

In [6]:
def runProject():
    # Load the training data you created
    with open('MyData/notes', 'rb') as filepath:
        notes = pickle.load(filepath)
    # Sort the pitches and store amount
    sortedPitches = sorted(set(item for item in notes))
    n_vocab = len(set(notes))
    # Build the melodies
    networkInput, inputAfterNormalization = constructMelodies(notes, sortedPitches, n_vocab)
    # Build the model
    model = constructNetwork(inputAfterNormalization, n_vocab)
    # Make the prediction
    actualPrediction = generate_next_note(model, networkInput, sortedPitches, n_vocab)
    # Create the file from the prediction
    buildMidiOutputFile(actualPrediction)

In [7]:
#print("start")
runProject()
#print("finish")

start
generate()
n_vocab =  186
Instructions for updating:
Colocations handled automatically by placer.
finish
